<a href="https://colab.research.google.com/github/AscharyaBalaji/runs/blob/master/Copy_of_whisper_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai-whisper pydub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 18.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/

In [ ]:
import whisper
import pandas as pd
import os
from tqdm import tqdm

In [ ]:
csv_path = "/content/whisper_final.csv"

In [ ]:
df = pd.read_csv(csv_path)


In [ ]:
if "audio_file" not in df.columns:
   raise ValueError("CSV file must contain a column named 'audio_file' with .mp3 filenames.")

In [ ]:
audio_folder = "/content"

In [ ]:
model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [00:47<00:00, 64.4MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [ ]:
# Function to transcribe audio
def transcribe_audio(file_path):
    try:
        result = model.transcribe(file_path)
        return result["text"]
    except Exception as e:
        return f"Error: {str(e)}"


In [ ]:
transcriptions = []
for audio_file in tqdm(df["audio_file"], desc="Transcribing Audio Files"):
    file_path = os.path.join(audio_folder, audio_file)
    if os.path.exists(file_path):
       transcription = transcribe_audio(file_path)
    else:
       transcription = "File not found"

    transcriptions.append(transcription)

df["transcription"] = transcriptions
output_csv_path = "/content/transcriptions.csv"
df.to_csv("transcriptions.csv", index=False)
print(f"Transcriptions saved to: {output_csv_path}")

Transcribing Audio Files: 100%|██████████| 147/147 [52:33<00:00, 21.45s/it]

Transcriptions saved to: /content/transcriptions.csv


In [ ]:
import pandas as pd

# Load transcriptions.csv
transcriptions = pd.read_csv("transcriptions.csv")

# Ensure audio_file is a string and remove .mp3
transcriptions["audio_file"] = transcriptions["audio_file"].astype(str).str.replace(".mp3", "", regex=False)

# Load the master file
master = pd.read_excel("trauma_llm_23_25_complete_master.xlsx")

# Ensure audio_file in master is also a string
master["audio_file"] = master["audio_file"].astype(str)

# Merge data on 'audio_file'
merged_df = master.merge(transcriptions, on="audio_file", how="left")

# Save the merged file
merged_df.to_excel("merged_trauma_llm.xlsx", index=False)

print("Merge complete. File saved as 'merged_trauma_llm.xlsx'.")


Merge complete. File saved as 'merged_trauma_llm.xlsx'.
